## Automated model defintion for variable global and local parameters
The overall goal of this structure is to have one top-level model which can be fed with the parameters relevant for a dataset. The model then passes the parameters necessary for the fitting of the individual experiments to models on the lower levels. Those calculate the scalar of the loss functions and the sum of all the loss functions is returned to the high-level model which then performs the optimization step.

### Importing packages

In [1]:
import logging
logger = logging.getLogger(__name__)
import matplotlib.cm
from matplotlib import pyplot
import numpy
import pandas
import pathlib
import scipy.optimize
import bletl
import murefi

### Create a dataset object from biolector data

In [2]:
filepath = pathlib.Path('Data', 'SiLA_Coryne_Standard_20181026_150350.csv')

data = bletl.parse(filepath) #dictionary with FilterTimeSeries objects

c:\users\helleckes\appdata\local\continuum\miniconda3\envs\master36\lib\site-packages\bletl\parsing\bl1.py:113: UserWarning: Calibration values for DO signal are missing. Skipping calibration.
  warnings.warn('Calibration values for DO signal are missing. Skipping calibration.')
c:\users\helleckes\appdata\local\continuum\miniconda3\envs\master36\lib\site-packages\bletl\parsing\bl1.py:104: UserWarning: Calibration values for pH signal are missing. Skipping calibration.
  warnings.warn('Calibration values for pH signal are missing. Skipping calibration.')


#### The parameter file determines the wells included in the dataset and fit
Here, only wells A2, A3, B1, B2 and B3 all having the same parameters are extracted from the data

In [3]:
parameter_path = pathlib.Path('Data', 'Parametertest.csv')

In [4]:
df_mapping = pandas.read_csv(parameter_path, sep=';').iloc[2:]

In [5]:
df_mapping

,Parameter,S_0,X_0,mue_max,K_S,Y_XS
2,A02,S_0,X_0,mue_max,K_S,Y_XS
3,A03,S_0,X_0,mue_max,K_S,Y_XS
4,B01,S_0,X_0,mue_max,K_S,Y_XS
5,B02,S_0,X_0,mue_max,K_S,Y_XS
6,B03,S_0,X_0,mue_max,K_S,Y_XS


In [6]:
class Model:
    def __init__(self):
        self.bounds = dict(
            S_0=(0,20),
            X_0=(0,1),
            mue_max=(0,0.8),
            K_S=(0,0.2),
            Y_XS=(0,0.8)
        )
        self.initial_guesses = dict(
            S_0=10,
            X_0=0.2,
            mue_max=0.3,
            K_S=0.05,
            Y_XS=(0.5)
        )
        return
    
model = Model()

In [7]:
mapping = murefi.ParameterMapping(df_mapping, bounds = model.bounds, guesses = model.initial_guesses)

In [8]:
mapping.mapping.keys()

dict_keys(['A02', 'A03', 'B01', 'B02', 'B03'])

In [9]:
def create_dataset_object(bletl_data, parameter_mapping):
    '''Function to create a dataset object for all replicates
    Args:
        bletl_data:     dictionary with 'FilterTimeSeries' objects (technicially 
                        calibrated data) from bletl (bldata.calibrated_data)
        par_dic:        dictionary containing well IDs as keys and the
                        parameters for fitting provided by the user
                        (murefi.ParameterMapping(parameter_path).parameters_dic)
    Returns: dataset objects (containing Replicate objects for each well)
    '''
    if not len(bletl_data['BS10'].value.keys()) == len(parameter_mapping.mapping.keys()):
        excluded = [
            key
            for key in bletl_data['BS10'].value.keys()
            if not key in parameter_mapping.mapping.keys()
        ]
        logger.warning(f'Not all measurements from {bletl_data} are converted to a Dataset object. Wells {excluded} were omitted.')
    dataset = murefi.Dataset()
    for well_id in parameter_mapping.mapping.keys():
        bs_x = bletl_data['BS10'].time[well_id][0:120]
        bs_y = bletl_data['BS10'].value[well_id].multiply(0.1187).add(0.5866)[0:120]
        bs_ts = murefi.Timeseries('BS', list(bs_x), list(bs_y))
        rep = murefi.Replicate(well_id)
        rep['BS'] = bs_ts
        dataset[well_id] = rep
    return dataset

    
dataset = create_dataset_object(data, mapping)
print('Wells included:', list(dataset.keys()))


Not all measurements from {'BS10': <bletl.core.FilterTimeSeries object at 0x00000252D295DBA8>} are converted to a Dataset object. Wells ['A01', 'A04', 'A05', 'A06', 'A07', 'A08', 'B04', 'B05', 'B06', 'B07', 'B08', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E07', 'E08', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08'] were omitted.


Wells included: ['A02', 'A03', 'B01', 'B02', 'B03']


### Function to calculate the loglikelihood of the dataset

## A fitting example

In [11]:
theta_guessed = mapping.guesses
model = murefi.MonodModel(['S','BS'])

In [12]:
dataset_objective = create_objective(dataset, model, mapping)

In [15]:
bounds = mapping.bounds
fit = scipy.optimize.minimize(murefi.objectives.for_dataset(dataset, model, mapping, error_models=[]), theta_guessed, bounds = bounds)

In [17]:
result = pandas.DataFrame(columns=['parameters', 'estimate'])
result['parameters'] = mapping.parameters
result['estimate'] = fit.x
result

,parameters,estimate
K_S,K_S,0.05
S_0,S_0,10.00
X_0,X_0,0.20
Y_XS,Y_XS,0.50
mue_max,mue_max,0.30


### Plotting
Plot the 5 replicates and the fit

In [21]:
mapping.order

('S_0', 'X_0', 'mue_max', 'K_S', 'Y_XS')

In [23]:
mapping.mapping

{'A02': ('S_0', 'X_0', 'mue_max', 'K_S', 'Y_XS'),
 'A03': ('S_0', 'X_0', 'mue_max', 'K_S', 'Y_XS'),
 'B01': ('S_0', 'X_0', 'mue_max', 'K_S', 'Y_XS'),
 'B02': ('S_0', 'X_0', 'mue_max', 'K_S', 'Y_XS'),
 'B03': ('S_0', 'X_0', 'mue_max', 'K_S', 'Y_XS')}

In [22]:
mapping.parameters

OrderedDict([('K_S', 'K_S'),
             ('S_0', 'S_0'),
             ('X_0', 'X_0'),
             ('Y_XS', 'Y_XS'),
             ('mue_max', 'mue_max')])

In [20]:
fit.x

array([ 0.05, 10.  ,  0.2 ,  0.5 ,  0.3 ])

In [18]:
#High resolution values for time
x_highres = numpy.arange(0, (dataset['A02'].x_max + 0.1), 0.1)

# Integrate data for the result of optimization
theta_fit = fit.x
y = model.solver(theta_fit[:2], x_highres, theta_fit)

#Plot replicates
pyplot.figure()
steps = numpy.arange(0, 1, 0.1).tolist()
cmap = pyplot.cm.coolwarm
for i, well in enumerate(dataset.keys()):
    pyplot.scatter(dataset[well]['BS'].x, dataset[well]['BS'].y, label=f'observed X, {well}', color=cmap(steps[i]))

#Plot fit
pyplot.plot(x_highres, y['BS'], label='log likelihood fit X', color='black')

pyplot.xlabel('time [h]')
pyplot.ylabel('CDW [g/L]')
_ = pyplot.legend()
pyplot.show()


ValueError: too many values to unpack (expected 3)